In [ ]:
################################################################################
#
####                               IMPORTING
#
################################################################################

#\\Standard libraries
from cmath import phase
from distutils.command.config import config
import pickletools
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle as pkl
import yaml
import os
from numpy import linalg as LA
import scipy.io as sio
import random

import sys
sys.path.append(os.path.dirname(os.getcwd()))

#\\Own libraries
from model.classic_detectors import *
from data.sample_generator import *
from utils.util import *
from runners.runner_langevin_real_data import *
from runners.runner_classic_methods_real_data import *
from pathlib import Path

In [ ]:
################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  Load parameters of the system ###
######################
dirPath = os.path.dirname(os.getcwd())
with open(dirPath + '/config_real_data.yml', 'r') as f:
    aux = yaml.load(f,  Loader=yaml.FullLoader)
config = dict2namespace(aux)    


######################
###  General setup ###
######################
SEED = 123
torch.manual_seed(SEED)
useGPU = False # If true, and GPU is available, use it.
LANGEVIN_DETECTOR = True
CLASSICAL_DETECTORS = True

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda:0'
else:
    device = 'cpu'

#\\\ Total number of data frames (200 max)
int_min = 0
int_max = 200

ofdmSymb = 0
subcarrierIndex = 0

#\\\ Index of data (allows to remove pilot signlas from array)
with open(dirPath + '/data/dataIndex', 'rb') as fp:
    dataIndex = pkl.load(fp)


In [ ]:
################################################################################
#
####                               MAIN
#
################################################################################



#\\\ Load data
subcarrier = dataIndex[subcarrierIndex]
batch_size, H_MMSE, H_comp, HPilotri, yPilotri, xri, xPilotri, pilotValue, pilotIndex, sigma, yri = loadData(ofdmSymb, subcarrier, subcarrierIndex, int_min, int_max,dirPath, dataIndex,config)

#\\\ Create generator and indices of the true symbols
generator = sample_generator(batch_size, config.mod_n, config.NR)
idx_real = generator.demodulate(xri)
j_indices = generator.joint_indices(idx_real)


if CLASSICAL_DETECTORS == True:
    #\\\ Run MMSE detector
    serMMSE, serSDR, x_MMSE = runClassicDetectors(config, generator, batch_size, device, H = H_MMSE, y = yri, noise_sigma = sigma  , j_indices = j_indices)
    
    #\\\ Phase correction
    samplesP = torch.zeros((batch_size, config.NT * 2, 4))
    index = 0
    #\\\ Pilot evaluation
    for pilotInd in range(4):
        serLangevin, serSDR, samplesP[:,:,index] = runClassicDetectors(config, generator, batch_size, device, H = HPilotri[:,:,:,index], y = yPilotri[index,:,:], noise_sigma = sigma, j_indices = j_indices)
        index =  index + 1

    samplesP_complexMMSE = samplesP[:, 0:8, :].cpu().detach().numpy() + 1j * samplesP[:, 8:, :].cpu().detach().numpy()
    phase_corr = samplesP_complexMMSE * np.conj(pilotValue)
    phase_err = np.angle(np.mean(phase_corr, 2))
    
    phase_compMMSE = np.exp(-1j*phase_err)

    #\\\ Here the phase correction is applied
    x_MMSE_complex = x_MMSE[:, 0:8].cpu().detach().numpy() + 1j * x_MMSE[:, 8:].cpu().detach().numpy()
    x_MMSE = np.multiply(x_MMSE_complex, phase_compMMSE)

    aux = torch.tensor(x_MMSE)
    xr = torch.real(aux)
    xi = torch.imag(aux)

    #\\\ Final estimated symbol in the real equivalent form
    x_MMSE_hat = torch.cat((xr, xi), dim=1)

if LANGEVIN_DETECTOR == True:

    #\\\ Phase correction
    samplesP = torch.zeros((batch_size, config.NT * 2, len(pilotIndex)))
    index = 0
    #\\\ Pilot evaluation
    for pilotInd in range(len(pilotIndex)):
        serLangevin, serSDR, samplesP[:,:,index] = runClassicDetectors(config, generator, batch_size, device, H = HPilotri[:,:,:,index], y = yPilotri[index,:,:], noise_sigma = sigma, j_indices = j_indices)
        index =  index + 1

    samplesP_complexMMSE = samplesP[:, 0:8, :].cpu().detach().numpy() + 1j * samplesP[:, 8:, :].cpu().detach().numpy()
    phase_corr = samplesP_complexMMSE * np.conj(pilotValue)
    phase_err = np.angle(np.mean(phase_corr, 2))

    phase_comp = np.exp(-1j*phase_err)

    #\\\ Multiply the channel by the phase correction before running the detection
    H = H_comp  * phase_comp[:,None,]
    H = torch.tensor(H)
    Hr = torch.real(H)
    Hi = torch.imag(H)
    h1 = torch.cat((Hr, 1. * Hi), dim=2)
    h2 = torch.cat((-1. * Hi, Hr), dim=2)
    H = torch.cat((h1, h2), dim=1)

    #\\\ Run langevin detector
    serLangevin, samples_hat, samples = runLangevin(config, generator, batch_size, device, H = H, y = yri, noise_sigma = sigma, j_indices = j_indices)


x_Lang_complex = samples_hat[:, 0:8].cpu().detach().numpy() + 1j * samples_hat[:, 8:].cpu().detach().numpy()

print(sym_detection(x_MMSE_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const))        
print(sym_detection(samples_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const))

In [ ]:
for user in range(0, config.NT):
    plt.figure(user)
    plt.plot(np.real(x_Lang_complex[:, user]), np.imag(x_Lang_complex[:, user]), 'xr')
    plt.plot(np.real(x_MMSE[:, user]), np.imag(x_MMSE[:, user]), 'xg')
    plt.plot(xri[:, user].cpu().detach().numpy(), xri[:, user + config.NT].cpu().detach().numpy(), 'ob')
    plt.axhline(y=0, color='k', linestyle='-')
    plt.axvline(x=0, color='k', linestyle='-')
    plt.xlabel('Re(x)', fontsize=14)
    plt.ylabel('Im(x)', fontsize=14)
    # plt.plot(np.real(samples[:, user]), np.imag(samples[:, user]), '*b')
    plt.legend(['Langevin', 'MMSE','True symbol'], loc = 1, fontsize=15)
    plt.tick_params(axis='both' , labelsize=13)
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.tight_layout()
    plt.show()
    # plt.savefig('user' + str(user) + '_withPhaseCorr_L.pdf')
    
#     # plt.show()
#     # plt.savefig('remote.pdf')

#     aa = np.zeros((config.n_sigma_gaussian * (config.n_sample_init+1), 2, 2*config.NT))

#     for ii in range(config.n_sigma_gaussian):
#         aa[0 + ii*(len(samples[ii])):(len(samples[ii])) + ii*(len(samples[ii])),:,:] = samples[ii]

#     import matplotlib.animation as animation
#     r1 = 0.15
#     r2 = 0.25
#     uu = (r1 - r2) * torch.rand(config.NT) + r2
#     uu = 0
#     initval = 0

#     fig = plt.figure()
#     plt.xlim([-1.5,1.5])
#     plt.ylim([-1.5,1.5])
#     user = 4
#     p, = plt.plot(aa[initval][0,user], aa[initval][0,user + config.NT], "o")
#     p2, = plt.plot(generator.QAM_const()[0] + uu,  generator.QAM_const()[1], "o", color='red')
#     p3, = plt.plot(aa[-1][0,0:config.NT], aa[-1][0,config.NT:], "o", color='green')
#     p5, = plt.plot(xri[0,0:config.NT] + uu, xri[0,config.NT:] + uu, "o", color='orange')
#     p4, = plt.plot(xri[0,user] + uu, xri[0,config.NT + user] + uu, "o", color='red')
# # p5, = plt.plot(samples_denoiserMMNet[initval][0,0:NT], samples_denoiserMMNet[initval][0,NT:], "o", color='orange')

#     def update(frame):
#     #     print(frame)
#         p.set_data(aa[frame + initval][0,user], aa[frame + initval][0,user + config.NT])
#         p2.set_data( generator.QAM_const()[0],  generator.QAM_const()[1])
#         p3.set_data(aa[-1][0,0:config.NT], aa[-1][0,config.NT:])
#         p5.set_data(xri[0,0:config.NT] + uu, xri[0,config.NT:] + uu)
#         p4.set_data(xri[0,user] + uu, xri[0,config.NT + user])
#     #     p5.set_data(samples_denoiserMMNet[frame + initval][0,0:NT], samples_denoiserMMNet[frame + initval][0,NT:])

#         return p, p2, p3, p4, p5

#     def init():
#         p.set_data(aa[initval][0,user], aa[initval][0,user + config.NT])
#         p2.set_data(generator.QAM_const()[0], generator.QAM_const()[1])
#         p3.set_data(aa[-1][0,0:config.NT], aa[-1][0,config.NT:])
#         p5.set_data(xri[0,0:config.NT] + uu, xri[0,config.NT:] + uu)
#         p4.set_data(xri[0,user] + uu, xri[0,config.NT + user])
#     #     p5.set_data(samples_denoiserMMNet[initval][0,0:NT], samples_denoiserMMNet[initval][0,NT:])
    
#         return  p, p2, p3, p4, p5


#     ani = animation.FuncAnimation(fig=fig, func=update, init_func=init,  frames=config.n_sigma_gaussian * (config.n_sample_init+1), interval=1)
#     plt.show()
#     ani.save('5.gif', fps=30)
